# Modèle Multi-encoder
Data Object Format:

    # Primaire
    "crt" : liste de token représentant la phrase courante
    "ctxs" : liste des phrases de contextes
    "matrices": liste des torch.Tensor des matrices d'attention entre la phrase courante et les phrases de contextes

    #Secondaire
    "matrices_suppr_uniforme": liste des torch.Tensor des matrices d'attention entre la phrase courante et les phrases de contextes dont les poids inférieur à une distribution uniforme sont supprimés.

    #Tertiaire
    "matrices_suppr_pad": liste des torch.Tensor des matrices d'attention entre la phrase courante et les phrases de contextes dont les poids correspondant aux tokens de padding sont supprimés.
    "crt_suppr_pad": liste de token réprésentant la phrase courante où les tokens de padding sont supprimés
    "ctxs_suppr_pad": liste des phrases de contextes dont les tokens de padding sont supprimés

    #Quaternaire
    "matrices_normalized": liste des torch.Tensor des matrices des poids d'attention entre la phrase courante et les phrases de contextes dont les poids sont normalisés à 1

    #Quintenaires
    "matrices_bpe_merged": liste des torch.Tensor des matrices d'attention entre la phrase courante et les phrases de contextes dont les poids correspondant à des BPEs sont fusionnés (gardant le max)
    "crt_bpe_merged": liste de token réprésentant la phrase courante où les tokens de BPEs sont fusionnés
    "ctxs_bpe_merged": liste des phrases de contextes dont les tokens de BPEs sont fusionnés


In [26]:
import Process_attention_weight as paw
import MultiEnc_utils as me
import torch

input_path_multienc=f"/home/getalp/lopezfab/lig/Attention/k3/Raw/raw_attn_1.json"

#######################################################################################
####################### PROCESS ATTENTION MODÈLE MultiEnc #############################
#######################################################################################

data = paw.lecture_data(input_path_multienc)

## On corrige les éléments reçus
data['crt'], data['ctxs'] = me.ajoute_eos_tokens(data['crt'], data['ctxs'])
for i in range(len(data["ctxs"])):
    data["matrices"][i] = torch.Tensor(data["matrices"][i])

## Suppression Inférieure à l'uniforme
data["matrices_suppr_uniforme"] = []
for i in range(len(data["ctxs"])):
    data["matrices_suppr_uniforme"].append(paw.suppression_inf_uniforme_tensor_src(data["matrices"][i]))


## Suppression Padding
data["ctxs_suppr_pad"]=[]
data["matrices_suppr_pad"]= []
for i in range(len(data["ctxs"])):
    temp_matrice, temp_crt, temp_ctx = paw.suppr_pad(_matrice= data["matrices_suppr_uniforme"][i], 
                _row_snt= data["crt"],
                _col_snt= data["ctxs"][i],
                strict= True)
    data["ctxs_suppr_pad"].append(temp_ctx)
    data["matrices_suppr_pad"].append(temp_matrice)
data["crt_suppr_pad"] = temp_crt

## Normalisation des matrices
data["matrices_normalized"]=[]
for i in range(len(data["ctxs"])):
    data["matrices_normalized"].append(paw.normalise_tensor(data["matrices_suppr_pad"][i]))

data["ctxs_bpe_merged"] = []
data["matrices_bpe_merged"] = []
for i in range(len(data["ctxs"])):
    temp_matrice, temp_crt, temp_ctxs = paw.fusion_bpe(_matrice= data["matrices_normalized"][i], 
                _row_snt= data["crt_suppr_pad"],
                _col_snt= data["ctxs_suppr_pad"][i]
                )
    data["ctxs_bpe_merged"].append(temp_ctxs)
    data["matrices_bpe_merged"].append(temp_matrice)
data["crt_bpe_merged"] = temp_crt

paw.affiche_matrix(paw.mise_en_forme_matricielle(data["matrices_bpe_merged"][0], data["crt_bpe_merged"], data["ctxs_bpe_merged"][0]), list)

print(data.keys())

['0-0', 'This', 'is', 'such', 'an', 'exciting', 'time', '.', '<eos>']
['Thank', 0.27651000022888184, 0.16107000410556793, 0.4301399886608124, 0.1472499966621399, 0.1859699934720993, 0.2626799941062927, 1.0, 0.6388400197029114]
['you', 0.2024800032377243, 0.20791999995708466, 0.330159991979599, 0.16986000537872314, 0.2704800069332123, 0.5195900201797485, 1.0, 0.8471099734306335]
['Simone', 0.24212999641895294, 0.24106000363826752, 0.4248799979686737, 0.21744999289512634, 0.41291001439094543, 0.5860499739646912, 1.0, 1.0]
[',', 0.17568999528884888, 0.1564200073480606, 0.1712699979543686, 0.13564999401569366, 0.24812999367713928, 0.5199599862098694, 1.0, 0.9563999772071838]
['Aly', 0.5457900166511536, 0.3031400144100189, 0.29907000064849854, 0.2417600005865097, 0.3589000105857849, 0.47391000390052795, 0.8590800166130066, 1.0]
[',', 0.20362000167369843, 0.17608000338077545, 0.2646999955177307, 0.12995000183582306, 0.23454000055789948, 0.38616999983787537, 1.0, 0.8096299767494202]
['Laurie'

In [ ]:
paw.fusion_bpe()

In [27]:
print(len(data["SL_matrice"]))
# len_crt_snt x x "Batch (128)"
temp = torch.Tensor(data["SL_matrice"])
print(temp.size())
print(len(data["crt"]))
print(len(data["crt_suppr_pad"]))
print(sum(len(i) for i in data["ctxs"]))
print(sum(len(i) for i in data["ctxs_suppr_pad"]))

73
torch.Size([73, 64, 512])
73
61
219
47


# Modèle par concaténation


data Object format:

    # Primaire
    "id": ID of the sentence,
    "src_tokens": full source sentence (context + current),
    "src_segments_labels": ,
    "enc_attn": attention matrix of the encoder,
    "tgt_tokens": full target sentence (context 9 current),
    "dec_attn": attention matrix of the decoder,

    # Secondaire
    "src_sentence": liste des phrases sources [k3, k2, k1, k0] où chaque phrase est une liste de token,
    "tgt_sentence": liste des phrases target [k3, k2, k1, k0] où chaque phrase est une liste de token,
    "enc_matrix": torch.DoubleTensor des poids d'attention de l'encoder,
    "dec_matrix": torch.DoubleTensor des poids d'attention du decoder,

    # Tertiaire
    "enc_matric_suppr_uinf_unif": torch.DoubleTensor des poids d'attention de l'encoder auquel ceux inférieur à une distribution uniforme sont nullifiés
    "dec_matric_suppr_uinf_unif": torch.DoubleTensor des poids d'attention du decoder auquel ceux inférieur à une distribution uniforme sont nullifiés
    "enc_matrices_attn": liste de dictionnaires contenant les matrices d'attention de l'encoder entre les phrases de contexte et courante.
    "dec_matrices_attn": liste de dictionnaires contenant les matrices d'attention du decoder entre les phrases de contexte et courante.
    
    


In [1]:
import Attention_utils as au
id = 4
input_path_concat=f"/home/getalp/lopezfab/lig/Attention/concat/Process/raw_json/{id}.json"
output_path_concat=f"/home/getalp/lopezfab/lig/Attention/concat1.1/Process/Output"

concat_processed = au.process_concat_matrice(absolute_input_path=input_path_concat)

In [10]:
#######################################################################################
######################### PROCESS ATTENTION MODÈLE CONCAT #############################
#######################################################################################

import torch
import copy
import Process_attention_weight as paw
import Concat_utils as cu
id = 4
path=f"/home/getalp/lopezfab/lig/Attention/concat/Process/raw_json/{id}.json"

data= paw.lecture_data(path)

## SRC side ############################################################################################################################################################
if type(data["src_tokens"]) == str:
    data["src_tokens"] = data["src_tokens"].split()

# On corrige les éléments reçus
data["src_tokens"] = paw.ajoute_eos_tokens_src(data["src_tokens"], \
                                               data["src_segments_labels"])

# On découpe la phrase complete dans les différentes phrases présentes
data["src_sentence"] = paw.full_sentence_to_ctx_and_crt(data["src_tokens"])
src_contextes = [range(len(data["src_sentence"]), 0, -1)]

# On converti la matrice en torch.DoubleTensor
data["enc_matrix"] = paw.convertion_matrice_str_to_float( data["enc_attn"])

# On supprime les poids inférieur à une distribution uniforme
data["enc_matrix_suppr_inf_unif"] = paw.suppression_inf_uniforme_tensor_src(data["enc_matrix"]) 

# Découpage de la matrice de self-attention en matrices de phrase x phrase
data["enc_matrices_attn"] = paw.cut_matrix_into_sentences(data["enc_matrix_suppr_inf_unif"], data["src_sentence"])

# Normalisation de chaque matrice phrase x phrase
data["enc_matrices_attn_normalized"] = []
for i in range(len(src_contextes)):
    temp = []
    for j in range(len(src_contextes)):
        temp.append({"crt": data["enc_matrices_attn"][i][j]["crt"],
            "ctx": data["enc_matrices_attn"][i][j]["ctx"],
            "matrix": paw.normalise_tensor(data["enc_matrices_attn"][i][j]["matrix"])})
    data["enc_matrices_attn_normalized"].append(temp)

# Fusion des BPEs pour chaque matrice phrase x phrase
data["enc_matrices_attn_normalized_bpe"] = []
for i in range(len(src_contextes)):
    temp = []
    for j in range(len(src_contextes)):
        temp.append({"crt": data["enc_matrices_attn_normalized"][i][j]["crt"],
            "ctx": data["enc_matrices_attn_normalized"][i][j]["ctx"],
            "matrix": paw.fusion_bpe(data["enc_matrices_attn_normalized"][i][j]["matrix"],
                                     data["enc_matrices_attn_normalized"][i][j]["crt"],
                                     data["enc_matrices_attn_normalized"][i][j]["ctx"])})
    data["enc_matrices_attn_normalized_bpe"].append(temp)



## TGT side ############################################################################################################################################################
if type(data["tgt_tokens"]) == str:
    data["tgt_tokens"] = data["tgt_tokens"].split()

# On corrige les éléments reçus
data["tgt_tokens"] = paw.ajoute_eos_tokens_tgt(data["tgt_tokens"], \
                                               data["tgt_eos_pos"])

# On découpe la phrase complete dans les différentes phrases présentes
data["tgt_sentence"] = paw.full_sentence_to_ctx_and_crt(data["tgt_tokens"])
tgt_contextes = len(data["tgt_sentence"])

# On converti la matrice en torch.DoubleTensor
data["dec_matrix"] = paw.convertion_matrice_str_to_float( data["dec_attn"])
data["dec_matrix"] = torch.where(data["dec_matrix"] == -1e8, torch.tensor(0.0), data["dec_matrix"]) # replace value of -1e8 to 0.0

# On supprime les poids inférieur à une distribution uniforme
data["dec_matrix_suppr_inf_unif"] = paw.suppression_inf_uniforme_tensor_tgt(data["dec_matrix"].transpose(0,1))

# Découpage de la matrice de self-attention en matrices de phrase x phrase
data["dec_matrices_attn"] = paw.cut_matrix_into_sentences(data["dec_matrix_suppr_inf_unif"], data["tgt_sentence"])

# Normalisation de chaque matrice phrase x phrase
data["dec_matrices_attn_normalized"] = []
for i in range(tgt_contextes):
    temp = []
    for j in range(tgt_contextes):
        temp.append({"crt": data["dec_matrices_attn"][i][j]["crt"],
            "ctx": data["dec_matrices_attn"][i][j]["ctx"],
            "matrix": paw.normalise_tensor(data["dec_matrices_attn"][i][j]["matrix"])})
    data["dec_matrices_attn_normalized"].append(temp)

# Fusion des BPEs pour chaque matrice phrase x phrase
data["dec_matrices_attn_normalized_bpe"] = []
for i in range(tgt_contextes):
    temp = []
    for j in range(tgt_contextes):
        _matrice, _row_snt, _col_snt = paw.fusion_bpe(data["dec_matrices_attn_normalized"][i][j]["matrix"],
                                     data["dec_matrices_attn_normalized"][i][j]["crt"],
                                     data["dec_matrices_attn_normalized"][i][j]["ctx"])
        temp.append({"crt": _row_snt,
            "ctx": _col_snt,
            "matrix": _matrice})
    data["dec_matrices_attn_normalized_bpe"].append(temp)


for i in range(tgt_contextes):
    for j in range(tgt_contextes):
        # print()
        paw.ecriture_tsv(matrice=data["dec_matrices_attn_normalized_bpe"][i][j]["matrix"],
                         crt=data["dec_matrices_attn_normalized_bpe"][i][j]["crt"],
                         ctx=data["dec_matrices_attn_normalized_bpe"][i][j]["ctx"],
                         id_crt=data["id"],
                         id_ctx=j,
                         absolute_folder_path="/home/getalp/lopezfab/Bureau/Attention/",
                         file_name=f"test_output_attn_{i}_{j}")

print(data)

4
4
{'id': 4, 'src_tokens': ['It', 'is', 'a', 'path@@', 'etic', 'little', 'second', '@-@', 'rate', 'substitute', 'for', 'reality', '--', 'a', 'pi@@', 'ti@@', 'able', 'flight', 'from', 'life', '.', '<eos>', 'As', 'such', ',', 'it', 'deserves', 'no', 'comp@@', 'as@@', 'sion', ',', 'it', 'deserves', 'no', 'treatment', 'as', 'minority', 'mar@@', 'ty@@', 'r@@', 'dom', ',', 'and', 'it', 'deserves', 'not', 'to', 'be', 'deemed', 'anything', 'but', 'a', 'per@@', 'ni@@', 'cious', 'si@@', 'ckness', '.', '&quot;', '<eos>', 'That', '&apos;s', 'from', 'Time', 'magazine', 'in', '1966', ',', 'when', 'I', 'was', 'three', 'years', 'old', '.', '<eos>', 'And', 'last', 'year', ',', 'the', 'president', 'of', 'the', 'United', 'States', 'came', 'out', 'in', 'favor', 'of', 'ga@@', 'y', 'marriage', '.', '<END>'], 'src_segments_labels': [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 

In [44]:
def ajoute_eos_tgt_tokens(_snt, tgt_segments_labels, eos_token = "<eos>"):
    """Ajoute un token end-of-sentence dans la phrase afin de faire concorder la taille de la matrice et de la phrase coté target

    Args:
        _snt (_type_): list de token de la phrase coté target
        tgt_segments_labels (_type_): list des position où ajouter le eos_token
        eos_token (str, optional): token end-of-sentence. Defaults to "<eos>".

    Returns:
        list: list de tokens de la phrase coté target avec les tokens end-of-sentence
    """
    snt= copy.deepcopy(_snt)
    for position in tgt_segments_labels:
        snt.insert(int(position[1:-1]), eos_token)
    return snt

my_tensor = torch.tensor([
    [0, 2, 3, 4],
    [1, 5, 6, 7],
    [2, 8, 9, 0]
])

ajoute_eos_tokens_src(["<pad>", "2", "4"], [1, 1, 2, 2])


['<pad>', '<eos>', '2', '4']

In [ ]:
print(data)

tensor([[ 1,  7,  9, 11, 13],
        [ 2,  8, 10, 12, 14],
        [ 8,  9,  5,  6, 11],
        [ 3,  6,  7,  8, 10]])
['Je', "m'", 'appelle', 'lipoupou']
['Je', "m'appelle", 'li', 'pou', 'pou']
